https://github.com/Kim-Ayeong/Fintech_Data_Analyst_Course/blob/master/Textbook5/notebook/1.%20n-Gram_200128.ipynb

# 준비

In [1]:
import pandas as pd
import numpy as np
import os
import re

import nltk
from numpy.random import randint, seed
from sklearn.feature_extraction.text import CountVectorizer

import konlpy
from konlpy.tag import Twitter

from collections import Counter
from wordcloud import WordCloud

from PIL import Image  
from IPython.display import set_matplotlib_formats
from nltk.corpus import stopwords

In [2]:
os.getcwd()

'C:\\workspace\\Python\\4-2) 핀테크 교육\\Final Project\\Project1'

In [3]:
os.chdir(r".\data\total")
os.getcwd()

'C:\\workspace\\Python\\4-2) 핀테크 교육\\Final Project\\Project1\\data\\total'

# Load data

In [4]:
my_text=pd.read_excel("교양오락문화생활비.xlsx", encoding='CP949')
my_text.shape

(13738, 3)

In [5]:
my_text['title'].values[:10]

array(['김준호 유튜브 채널 구독자 38만명 돌파…뼈그맨 인증',
       '\'두데\' 감스트 "구독자들 \'야\'라고 불러…난 \'인직아\'라고 불린다"',
       '[뉴스레터 구독 이벤트] 1차 당첨자 추첨 과정 공개…100명의 행운의 주인공은...',
       '[MT리포트] "1년 수입 10억"…수백만 구독자 \'유튜버의 세계\'',
       'CJ E&M, ‘다이아 티비’ 구독자 1억 6000만명 돌파',
       "'슈가맨' '히든싱어' 등 음악예능 인기에…JTBC 유튜브 구독자 200만 돌파",
       "250만 구독자 '유튜버 망치'한국의 맛 알려",
       "[뉴스레터 구독 이벤트] RC카 '러커스'로 대리만족 한 번 해보시렵니까",
       "[뉴스레터 구독 이벤트] 1차를 놓쳤다면 'V30'가 기다리는 2차 이벤트에 도전",
       '250만 구독자 유튜버가 전하는 한국의 맛'], dtype=object)

# 전처리

In [6]:
len(my_text['title'].values) - len(set(my_text['title'].values)) # 중복 데이터

772

In [7]:
no_meaning = "구독|책|핫플레이스|핫|플레이스|레이스|휴가비"
my_text_clean = []

for a_line in set(my_text['title'].values):
    a_line = re.sub('[^ㄱ-ㅣ가-힣]+', ' ', a_line) # 한글, 숫자 이외는 모두 공백으로
    a_line = re.sub(no_meaning, '', a_line)           # 의미없는 문자 삭제
    a_line = re.sub('\s+', ' ', a_line)               # 잉여 스페이스를 공백 하나로
    my_text_clean += [a_line]

In [8]:
my_text_clean[:10]

['천년의 숲 비자림과 국내 최고 당근 마을',
 ' 생활건강 온더바디 새 모델에 배우 유인나 발탁',
 ' 의해 기념 북콘서트 나는 기억이다 ',
 ' 년 섭외 순위 박나래의 활약이 기대되는 이유',
 ' 읽는 세종 올해도 만 양서 확충',
 '어린이날 행사 풍성 평화가족한마당',
 '한해 양다일 연애포차 사실은 일 공개',
 ' 골목식당 백종원 국수집 이을 대환장 식당 등장 폐업까지 고려 ',
 '전북대 학생들 전북 전승설화 조사해 쓰다',
 ' 힘들때 손 내미는 게 진정한 후원 박인비처럼 골프도 스타 키울 것 ']

In [9]:
hannanum = konlpy.tag.Hannanum()

# 명사 추출
my_words = []
for a_line, n in zip(my_text_clean, range(len(my_text_clean))):
    #print(n, end=" ")
    if a_line == " ":
        #print("패스")
        continue
    my_words += hannanum.nouns(a_line)
    
#my_words[:10]

In [10]:
# 단음절 제거
my_words_2 = [""]
for a_word in my_words:
    if len(a_word) > 1:
        my_words_2[0] += " " + a_word
        
#my_words_2

# 2-gram

In [11]:
n = 2                                                            # Can be changed to a number equal or larger than 2.
n_min = n
n_max = n
n_gram_type = 'word'                                             # n-Gram with words.
vectorizer = CountVectorizer(ngram_range=(n_min,n_max), analyzer = n_gram_type)

In [12]:
n_grams = vectorizer.fit(my_words_2).get_feature_names()            # Get the n-Grams as a list.
n_gram_cts = vectorizer.transform(my_words_2).toarray()             #  The output is an array of array.
n_gram_cts = list(n_gram_cts[0])                                 # Convert into a simple list.

In [13]:
res_dict = {}
for z, zz in zip(n_grams, n_gram_cts):
    res_dict[z] = zz
res_dict = sorted(res_dict.items(), key=(lambda x: x[1]), reverse=True)
res_dict[:30]

[('정규직 전환', 103),
 ('국정원 특활비', 73),
 ('근로자 휴가지원사업', 60),
 ('만명 돌파', 56),
 ('임금협약 체결', 50),
 ('박서준 박민영', 42),
 ('김비서 박서준', 35),
 ('박근혜 국정원', 34),
 ('노사 임단협', 33),
 ('만원 지원', 33),
 ('정규직 정규직', 33),
 ('한국 노사', 33),
 ('서울시장 후보', 32),
 ('국정원 뇌물', 28),
 ('이민우 여진구', 28),
 ('문고리 인방', 26),
 ('캘린더 서비스', 25),
 ('밤도깨비 김지원', 24),
 ('삼성전자 갤럭시', 23),
 ('근로자 만원', 22),
 ('구구단 세정', 21),
 ('박근혜 대통령', 21),
 ('우리 동네', 21),
 ('짠내투 김생민', 21),
 ('근로자 정규직', 20),
 ('근로자 휴가지원', 20),
 ('포토 아이들', 20),
 ('홍석천 이민우', 20),
 ('보이스피싱 인출', 19),
 ('신문 취소', 19)]

In [14]:
n_grams = vectorizer.fit(my_words_2).get_feature_names()           # A list of n-Grams.
my_dict = {}
for a_gram in n_grams:
    words = nltk.word_tokenize(a_gram)
    a_nm1_gram = ' '.join(words[0:n-1])                         # (n-1)-Gram.
    next_word = words[-1]                                       # Word after the a_nm1_gram.
    if a_nm1_gram not in my_dict.keys():
        my_dict[a_nm1_gram] = [next_word]                       # a_nm1_gram is a new key. So, initialize the dictionary entry.
    else:
        my_dict[a_nm1_gram] += [next_word]                      # an_nm1_gram is already in the dictionary.

# 다음 연관 단어 확인

In [15]:
def next_word (arr):
    for i in arr:
        print(i, "--->", my_dict[i][:5])

In [16]:
next_word(['여행', '맛집', '종합', '오늘'])

여행 ---> ['가성비', '가이드', '가즈아', '가지', '강원']
맛집 ---> ['가빈', '가심비', '각종', '갈비씨', '갈비찜']
종합 ---> ['가성비', '강원', '검찰', '경부선', '경주']
오늘 ---> ['개통', '걸음', '결론', '결정', '경주']
